# Code Generator

The requirement: use a Frontier or free model to generate high performance new code from original one


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4o-mini and Claude-3-Haiku, which are the slightly lower priced models, and two other free models, which I host in Ollama client but which can be reached with external endpoints or changed.
                The program allow to import code from a file or pasting it into the textbox of Original code. The language of this code can be choosen among the dropdown list and, similarly, the conversion code language can be choosen among another dedicated dropdown.
                The code can be tested in an online platform like w3cschool or similar. There will be surprises about the quality of code production between thedifferent models!
                <br>
                Try to sumbit a code generated by a model to another model, same language, the conversion should try only to optimize the code . See the result.
            </span>
        </td>
        
        
            
    
</table>

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()
qwencoder= OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
phi4= OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

PHI4_MODEL ='phi4'
QWENCODER_MODEL = 'ticlazau/qwen2.5-coder-7b-instruct'
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"


In [4]:
def system_messages(original_lang,lang):
    system_message=f"""
    You are an assistant that reimplements {original_lang} code  in high performance {lang} for a win64 pc. 
    Respond only with {lang} code; use  only  proper code comments. Do not include anything outside strict coding text as response.thing 
    Mark the beginning of code with "<START>" as the first line and with <END> as the last line. Cut everything before and after those markers and
    do not include anything which is not code between those markers.
    The {lang} response needs to produce an identical output in the fastest possible time. 
    Important note: if the original language and destination languages are the same, try to debug the code and output a better version of it.
    """
    return system_message

In [5]:
def user_prompt_for(original_code,lang):
    user_prompt = f"Rewrite the following code in {lang} with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with strict {lang} code; do not explain your work or add any comment outside code body.  "
    user_prompt += f"Pay attention to number types to ensure no int overflows. Remember to include all necessary {lang} packages needed. This is the code:\n\n"
    user_prompt += original_code
    return user_prompt

In [6]:
def messages_for(original_code,lang,original_lang):
    return [
        {"role": "system", "content": system_messages(lang,original_lang)},
        {"role": "user", "content": user_prompt_for(original_code,lang)}
    ]

In [7]:
# write to a file called optimized.xxx

def write_output(reply,lang):
     
    if len(reply)==0:
        return 'Failed to save 0 length file'
    
    match lang:
        case'C#':
            filename ='optimised.cs'
        case 'Rust':
            filename ='optimised.rs'
        case'Go':
            filename='optimised.go'
        case'C++':
            filename= 'optimised.cpp'
        case 'Python':
            filename='optimised.py'
        case 'Php':
            filename='optimised.php'
        case 'Perl':
            filename='optimised.pl'
        case _ :
            return 'Failed to save wrong extension'
    
    try:
        with open(filename, "w") as f:
            f.write(reply)
            return 'File saved'
    except:
        return 'Failed to save file'
    

<b>The code below is left just as example</b>

In [8]:
original_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [9]:
def stream_gpt(original_code,lang,original_lang):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [10]:
def stream_qwen(original_code,lang,original_lang):    
    stream = qwencoder.chat.completions.create(model=QWENCODER_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [11]:
def stream_phi4(original_code,lang,original_lang):    
    stream = phi4.chat.completions.create(model=PHI4_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [12]:
def stream_claude(original_code,lang,original_lang):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_messages(lang,original_lang),
        messages=[{"role": "user", "content": user_prompt_for(original_code,lang)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('<START>','').replace('<END>','')

In [13]:
def optimize(original_code, model,lang,original_lang):
    if model=="GPT":
        result = stream_gpt(original_code,lang,original_lang)
    elif model=="Claude":
        result = stream_claude(original_code,lang,original_lang)
    elif model == 'Phi4':
        result = stream_phi4(original_code,lang,original_lang)
    else:
        result = stream_qwen(original_code,lang,original_lang)
    for stream_so_far in result:
        yield stream_so_far        

In [14]:
def upload_code(filename):
    #print(filename)
    with open(filename,'r') as file:
        code= file.read()
    #print(code)
    return code

In [31]:
css = """
.convert {background-color: #306998;}
.save {background-color: green;}
.clear{background-color: orange;}
"""

In [32]:
with gr.Blocks(css=css) as ui:
    lang =''
    original_lang=''
    original_code=''
    
    with gr.Row():
          
        original_code = gr.Textbox(label="Original code:", lines=10,) 
        code = gr.Textbox(label=f"{lang} Converted code:", lines=10)
    with gr.Row():
        original_lang=gr.Dropdown(['C++','C#','Rust','Go', 'Python','Php','Perl'],label='Select original language')        
        lang = gr.Dropdown(['C++','C#','Rust','Go', 'Python','Php','Perl'],label='Select new language', value ='C++')
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(["GPT", "Claude",'Phi4',"Qwen"], label="Select model", value="Qwen")          
            uploadBtn= gr.UploadButton('Import code',elem_classes=['convert'])
            clearBtn =gr.Button('Clear',elem_classes=['clear'])
            
        with gr.Column():
            convert = gr.Button("Convert code",elem_classes=['convert'])
            saveBtn=gr.Button('Save to a file',elem_classes=['save'])
            saving=gr.Textbox(label='File save result')

    uploadBtn.upload(upload_code,uploadBtn, outputs=original_code)
    convert.click(optimize, inputs=[original_code, model,lang,original_lang], outputs=[code])
    saveBtn.click(write_output,inputs=[code,lang],outputs=[saving])
    clearBtn.click(lambda: [None,None], outputs=[original_code,code])
    
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
